 # This is an Exemple on how Can Programming help Teaching in Chemistry

Through this exemple, I shall show how the teaching of kinetics can be improved using interactive schemes and simulations. 


In [7]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt
import time
import numpy as np

## First Order Kinetics: A$\rightarrow$ B
Lets start by seeing the time evolution of a simple $1^{st}$ order reaction such as A $\rightarrow$ B. Lets assume it as a first order, which means that the rate of reaction is proportional to the amount of reactant. In other words:  
 $$\frac{dB}{dt} = k_1A$$ 
 Due to mass concervation (A turns into B) the rate of generation of A is equal to the rate of consumption of B, meaning: 
 $$\frac{dA}{dt} = -\frac{dB}{dt} = -k_1A$$ 
 The following cells evaluate the evolution of the concentration of A with respect to time. The first cell is used to define the numerical method to solve the equation (you can ignore it at first, but execute it!)


In [8]:
NT = 100;
T = 2;
dt = T/NT;

def FirstOrder_RKtev1(A0,B0,dt,k_1): #Defines R-K time evolution step.
    h1 = k_1*(A0);
    h2 = k_1*(A0+dt*h1/2);
    h3 = k_1*(A0+dt*h2/2);
    h4 = k_1*(A0+dt*h3);
    A1 = A0 +dt/6*(h1+2*h2+2*h3+h4);
    B1 = B0 -dt/6*(h1+2*h2+2*h3+h4);
    return A1,B1;

def fsimple(A0,B0,k_1):
    for ki in range(0,NT):
        A1,B1 = FirstOrder_RKtev1(A0,B0,dt,-k_1);
        plt.plot(ki*dt,A0,'ro');
        plt.plot(ki*dt,B0,'bo');
        A0 = A1
        B0 = B1;
    plt.xlabel('time(s)');
    plt.ylabel('Concentration (mol/L)');
    plt.legend(['A','B']);
    return 

def fanalytical(A0,B0,k_1):
    t = np.arange(NT)*dt;
    A = A0*np.exp(-k_1*t);
    B = (A0+B0)-A;
    plt.plot(t,A,'r');
    plt.plot(t,B,'b')
    plt.xlabel('time(s)');
    plt.ylabel('Concentration (mol/L)');
    plt.legend(['A','B']);

In [9]:
interact_manual(fsimple,A0=(0,10),B0=(0,10),k_1=(0.0,10.0));

interactive(children=(IntSlider(value=5, description='A0', max=10), IntSlider(value=5, description='B0', max=1…

Exercises:

$\bullet$ Play with the initial concentrations of A and B (parameters A0 and B0). If more B is present at the beginning, $B0 \neq 0 $ what is the effect on the final concentrations? 

$\bullet$ Change the kinetic rate (k_1) to 0.1 s$^{-1}$. Within the timescale of the plot above (5s) , do you think the reaction is over? Why? 

$\bullet$ Now, solve the equations analytically and plot it in the graph (replacing the last line of the code by your solution).

# First Order Kinetics, a reversible reaction: A$\rightleftharpoons$ B
Now lets add the reverse reaction. B$\rightarrow$A. The easier way of looking at this reaction is by writing it as its elemental steps:
$$A\xrightarrow{k_1}B $$ 
$$B\xrightarrow{k_{-1}}A$$

Now, lets write the rate of consumption of A. 
$$\frac{dA}{dt}= \underbrace{-k_1A}_{\text{from the first equation}} + \underbrace{k_{-1}B}_{\text{from the second equation}}$$

In order to solve this equation, we must now express B in terms of A. The mass concervation law states will be helpful: 

$\frac{d\text{(total number of molecules)}}{dt} = 0$

Or, equivalently: 

$\frac{d(A+B)}{dt}= 0 \Rightarrow \frac{dA}{dt}+ \frac{dB}{dt}= 0 \Rightarrow \frac{dA}{dt} = -\frac{dB}{dt}$


In [10]:
def FirstOrder_RKtev2(C0,dt,k): #Defines R-K time evolution step.
    h1 = np.dot(C0,k); #print(h1);
    h2 = np.dot(C0+dt*h1/2,k);
    h3 = np.dot(C0+dt*h2/2,k);
    h4 = np.dot(C0+dt*h3,k); #print(h4);
    A1,B1 = C0 + dt/6*(h1+2*h2+2*h3+h4);
    return A1,B1;

def frev(A0,B0,k_1,km1):
    C0 =np.array([A0,B0]);
    k = np.array([[-k_1,k_1],[km1, -km1]]);
    for ki in range(0,NT):
        A1,B1 = FirstOrder_RKtev2(C0,dt,k);
        plt.plot(ki*dt,A0,'ro');
        plt.plot(ki*dt,B0,'bo');
        A0 = A1
        B0 = B1;
        C0 =np.array([A0,B0]);
    plt.xlabel('time(s)');
    plt.ylabel('Concentration (mol/L)');
    plt.legend(['A','B']);

NT = 200;
T = 5;
dt = T/NT;



In [11]:
interact_manual(frev,A0=(0,10),B0=(0,10),k_1=(0.0,10.0),km1 = (0.0,10.0));

interactive(children=(IntSlider(value=5, description='A0', max=10), IntSlider(value=5, description='B0', max=1…

Exercises: 

$\bullet$ Keep $k_1$ constant and vary the value of $k_{-1}$. What do you observe?

$\bullet$ Set $k_1 = k_{-1} = 1$, then $k_1 = k_{-1} = 10$. Can you explain such change in the graph?

$\bullet$ Set $k_1 = k_{-1} = 3$. When is it reasonable to assume the system has reached a stationnary state?

$\bullet$ Solve the equations analytically and plot it replacing the last line of the code. 

$\bullet$ Can you predict the stationary state concentration ratio directly from the reactions constants ($k_1$ and $k_{-1}$)?


# Multistep Reactions

Now, lets analyze the following mechanism:
$$A\rightleftharpoons B$$
$$B\rightarrow C $$
$$C \rightleftharpoons D $$ 

Again, this is easier written as a sequence of elementary steps:

\begin{align}
A & \xrightarrow{k_1} B\\
B & \xrightarrow{k_{-1}} A \\
B & \xrightarrow{k_2} C\\
C & \xrightarrow{k_{3}}D\\
D & \xrightarrow{k_{-3}} C
\end{align}







In [14]:
NT = 200;
T = 10;
dt = T/NT;

def FirstOrder_RKtev3(C0,dt,k): #Defines R-K time evolution step.
    h1 = np.dot(C0.T,k); #print(h1);
    h2 = np.dot(C0+dt*h1/2,k);
    h3 = np.dot(C0+dt*h2/2,k);
    h4 = np.dot(C0+dt*h3,k); #print(h4);
    C1 = C0 + dt/6*(h1+2*h2+2*h3+h4);
    return C1;

def fmultstep(A0,B0,C0,D0,k_1,km1,k_2,k_3,km3):
    Cond0 = np.array([A0,B0,C0,D0]);
    k = np.array([[-k_1,km1,0,0],[k_1,-km1-k_2,0,0],[0,k_2,-k_3,km3],[0,0,k_3,-km3]]);
    for ki in range(0,NT):
        Cond = FirstOrder_RKtev3(Cond0,dt,k.T);
        plt.plot(ki*dt,Cond[0],'ro');
        plt.plot(ki*dt,Cond[1],'bo');
        plt.plot(ki*dt,Cond[2],'go');
        plt.plot(ki*dt,Cond[3],'ko');
        Cond0 = Cond
    plt.xlabel('time(s)');
    plt.ylabel('Concentration (mol/L)');
    plt.legend(['A','B','C','D']);
    return;

In [15]:
interact_manual(fmultstep,A0=(0,2.),B0 = (0,2.),C0 =(0,2.),D0 =(0,2.),k_1=(0,10.),km1 = (0,10.),k_2=(0,10.),k_3=(0,10.),km3 = (0,10.));

interactive(children=(FloatSlider(value=1.0, description='A0', max=2.0), FloatSlider(value=1.0, description='B…